# Text Parser Testing document

This document is going to read the text file found in ../generated_data/Logs2020OCR_avepdf.com_horizontal.txt which was generated using the following website:


[avepdf.com](http://avepdf.com/)

In [ ]:
import re # Regulra expresion library

In [ ]:
#filename='../generated_data/Logs2020OCR_avepdf.com_horizontal.txt'
filename='../generated_data/Logs2019OCR_avepdf.com_horizontal.txt'

In [ ]:
# read file in as a list of lines
lines = []
with open(filename) as f:
    lines = f.readlines()
    
for ind in range(len(lines)):
    lines[ind] = lines[ind].lstrip().rstrip()

In [ ]:
#lines

In [ ]:
# Parse cells using 20-1 regular expresisons
# Result is a list of list where each call is a list of lines for the call.
calls = []

call = []
for line in lines:
    if line == '':
        pass
    else:
        if re.match("[1-2][9,0]-[0-9]+\s", line[:10]):
            if len(call) > 0:
                calls.append(call)
                call = []
        call.append(line)
if len(call) > 0:
    call.append(line)

In [ ]:
len(calls)

In [ ]:
calls[-1]

In [ ]:
def parseHeader(line):
    print(line)
    index_past = 0
        
    callNumberMatch = re.search("[19,20]-\d+", line)
    if callNumberMatch is not None:
        callNumber = callNumberMatch.group()
        index_past = callNumberMatch.span()[1] + 1
    else:
        callNumber = None
        
    timeMatch = re.search("\d\d\d\d", line[index_past:])
    if timeMatch is not None:
        callTime = timeMatch.group()
        index_past += timeMatch.span()[1]+1
    else:
        callTime = None
        
    spaces = 0
    index = len(line)-1
    while spaces < 8:
        if line[index] == " ":
            spaces += 1
        else:
            spaces = 0
        index -= 1
    callReason = re.sub("\s+", " ", line[index_past:index])
    if callReason[0] == " ":
        callReason = callReason[1:]
    if callReason[-1] == " ":
        callReason = callReason[:-1]
    callAction = re.sub("\s+", " ", line[index+8:])
    if callAction[0] == " ":
        callAction = callAction[1:]
    if callAction[-1] == " ":
        callAction = callAction[:-1]
        
    return [callNumber, callTime, callReason, callAction]

def get_unit_times(unit_str):
    print(unit_str)
    times = re.sub(' +', ' ',unit_str).split(' ')
    tm_dict = {}
    for tm in times:
        if '-' in tm:
            vals = tm.split('-')
            tm_dict[vals[0]] = vals[-1]
    return tm_dict

def parse_call_list(call):
    '''Parse a call list into a dictionary'''
    
    if len(call) == 0:
        return
    
    my_call = {}
    
#     header = parseHeader(call[0])
#     my_call['callNumber'] = header[0]
#     my_call['callTime'] = header[1]
#     my_call['callReason'] = header[2]
#     my_call['callAction]']= header[3]
    my_call['header'] = call[0]
    
    

    
    ind = 0
    individual = ''
    while ind < len(call):
        line = call[ind]
        myline = line.split(':')
        if len(myline) == 2:
            tag = myline[0].rstrip().lstrip()
            tag = re.sub(' +', ' ',tag)
            value = myline[1].rstrip().lstrip()
            if tag == 'Narrative':
                if tag in my_call:
                    narrative = my_call['Narrative']
                else:
                    narrative = ''
                ind +=1
                while ind < len(call):
                    myline = call[ind].split(':')
                    if len(myline) == 1:
                        narrative += re.sub(' +', ' ',call[ind]) + " "
                    ind += 1
                my_call['Narrative'] = narrative
            else:
                if tag == "Unit":
                    ind += 1;
                    if ind < len(call):
                        unit_info = get_unit_times(call[ind])
                        if 'Units' in my_call:
                            my_call['Units'].append((value, unit_info))
                        else:
                            my_call['Units'] = [(value, unit_info)]
                else:
                    if tag == "Operator" or tag == "Owner":
                        individual=tag+"_"
                    my_call[tag] = value
        else:
            if len(myline) > 2:
                tags = re.findall('[\S]+:', line)

                for tag in reversed(tags):
                    start = line.rindex(tag)
                    value = line[start+len(tag):].rstrip().lstrip()

                    if len(value) > 0:
                        tag = tag[:-1]
                        my_call[individual+tag] = value
                        line = line[:line.rindex(tag)]
        ind +=1
    return(my_call)

In [ ]:
#Parse an individual call
parse_call_list(calls[44])

In [ ]:
#Parse all of the calls
call_dicts =[]
unit = ''
for call in calls:
    my_call = parse_call_list(call)
    call_dicts.append(my_call)
    print(my_call['header'])

In [ ]:
call_dicts[1]

In [ ]:
total_stops = 0
tot = 0
total_race = {}
norace = 0
my_stops = []
for call in call_dicts:
    if 'Call Taker' in call:
        if "craig" in call['Call Taker'].lower():
            tot += 1
            #print(call['header'])
            if "STOP" in call['header']:
                my_stops.append(call)
                total_stops += 1
                if 'Operator_Race' in call:
                    race = call['Operator_Race'].lower()
                    if race in total_race:
                        total_race[race] = total_race[race]+1
                    else:
                        total_race[race] = 1
                else:
                    norace +=1

In [ ]:
tot

In [ ]:
total_stops # by hammer

In [ ]:
total_race

In [ ]:
my_stops

In [ ]:
# # Print out only the narratives

# for call in call_dicts:
#     if "Narrative" in call:
#         print(call['Narrative'])
#         print('\n')

In [ ]:
from ipywidgets import interact
# use interact decorator to decorate the function, so the function can receive the slide bar's value with parameter x.
@interact(x=(0, 100))
def double_number(x):
    print("The double of %d is %d." % (x, x*2))
    
# add another slide bar to this function with the interact decorator.    
@interact(y=(0, 10))
def square_number(y):
    print("The square of %d is %d." % (y, y**2))